### Step1: Load dataset

In [30]:
import pandas as pd
import numpy as np

# Path to extracted TXT file
file_path = 'household_power_consumption.txt'

# Load with ; separator, handle '?' as NaN
data = pd.read_csv(file_path, sep=';', na_values='?')

# Combine Date and Time to datetime
data['datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d/%m/%Y %H:%M:%S')

# Numerical timestamp: seconds since the earliest date
min_dt = data['datetime'].min()
data['timestamp'] = (data['datetime'] - min_dt).dt.total_seconds()

# Relevant columns (drop Date/Time/datetime, keep numerics)
cols = ['timestamp', 'Global_active_power', 'Global_reactive_power', 'Voltage', 
        'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
data = data[cols].dropna()  # ~1.25% missing, drop for simplicity
full_data_size = len(data)

print(f"Dataset loaded: {data.shape[0]} rows")
print(f"First 5 data: {data.head()}")
print(f"Data size: {full_data_size}")


Dataset loaded: 2049280 rows
First 5 data:    timestamp  Global_active_power  Global_reactive_power  Voltage  \
0        0.0                4.216                  0.418   234.84   
1       60.0                5.360                  0.436   233.63   
2      120.0                5.374                  0.498   233.29   
3      180.0                5.388                  0.502   233.74   
4      240.0                3.666                  0.528   235.68   

   Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
0              18.4             0.0             1.0            17.0  
1              23.0             0.0             1.0            16.0  
2              23.0             0.0             2.0            17.0  
3              23.0             0.0             1.0            17.0  
4              15.8             0.0             1.0            17.0  
Data size: 2049280


### Step2: Create a Small Offline Sample

In [31]:
# Function for sampling-based approximate SUM (scaled)
def sample_sum(query, samp_df, full_size):
    mask = np.ones(len(samp_df), dtype=bool)
    for dim, (lower, upper) in query.items():
        mask &= (samp_df[dim] >= lower) & (samp_df[dim] <= upper)
    subset_sum = samp_df.loc[mask, agg_col].sum()
    scale = full_size / len(samp_df)
    return subset_sum * scale

In [32]:
sample_size = int(0.001 * len(data))  # ~2000 rows
sample = data.sample(n=sample_size, random_state=42).copy()
print(f"Sample created: {sample.shape[0]} rows")

Sample created: 2049 rows


### Step 3: Generate a Historical Query Log

In [33]:
import random

# Define dimensions (7D) and their min/max
dimensions = ['timestamp', 'Global_reactive_power', 'Voltage', 'Global_intensity', 
              'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
dim_ranges = {dim: (data[dim].min(), data[dim].max()) for dim in dimensions}

# Aggregate column
agg_col = 'Global_active_power'

# Add this after loading data
quantiles_low = data[dimensions].quantile(0.05)
quantiles_high = data[dimensions].quantile(0.95)

def generate_random_query_paper_style():
    """
    Generate query ranges exactly as described in the LAQP paper for POWER dataset:
    - Left boundary: uniform from [min, min + (max-min)/4]  → first quarter
    - Right boundary: uniform from [max - (max-min)/4, max] → last quarter
    This ensures lower < upper with high probability and avoids empty results.
    """
    predicates = {}
    for dim in dimensions:
        min_val = data[dim].min()
        max_val = data[dim].max()
        range_width = max_val - min_val
        
        # First quarter for left boundary
        left_min = min_val
        left_max = min_val + range_width / 4
        lower = random.uniform(left_min, left_max)
        
        # Last quarter for right boundary
        right_min = max_val - range_width / 4
        right_max = max_val
        upper = random.uniform(right_min, right_max)
        
        # Ensure lower < upper (very rare failure, but safe)
        if lower >= upper:
            lower, upper = right_min, right_max  # fallback
        
        predicates[dim] = (lower, upper)
    
    return predicates

# Function to compute exact SUM for a query
def exact_sum(query, df):
    mask = np.ones(len(df), dtype=bool)
    for dim, (lower, upper) in query.items():
        mask &= (df[dim] >= lower) & (df[dim] <= upper)
    return df.loc[mask, agg_col].sum()

# For training: 2000 queries (as in paper)
query_log = []
num_train_queries = 2000
attempts = 0
max_attempts = 20000

print("Generating 2000 training queries (paper style)...")
while len(query_log) < num_train_queries and attempts < max_attempts:
    q = generate_random_query_paper_style()
    exact_result = exact_sum(q, data)
    
    if exact_result > 0.01 * avg_exact:  # safe threshold > 0
        estimate = sample_sum(q, sample, full_data_size)
        error = exact_result - estimate
        query_log.append({'query': q, 'exact': exact_result, 
                          'estimate': estimate, 'error': error})
    attempts += 1

print(f"Generated {len(query_log)} training queries in {attempts} attempts")

# Retrain the model on these 800
# (same as before: prepare X, y from query_log, fit model)

Generating 2000 training queries (paper style)...
Generated 2000 training queries in 2000 attempts


In [34]:
print(query_log[0])
print(query_log[100])
print(query_log[200])
print(query_log[300])
print(query_log[500])

{'query': {'timestamp': (27585693.219044786, 116127543.95249799), 'Global_reactive_power': (0.32342049411539786, 1.1483623059071757), 'Voltage': (229.14521583595882, 248.84322300459831), 'Global_intensity': (3.1926344081068048, 36.965980921418), 'Sub_metering_1': (12.079002979585413, 75.26493117494343), 'Sub_metering_2': (4.665683772586037, 69.99493896258798), 'Sub_metering_3': (7.5747769122624975, 29.19202641833582)}, 'exact': 3840.198, 'estimate': 0.0, 'error': 3840.198}
{'query': {'timestamp': (6899750.711142928, 120786408.4130603), 'Global_reactive_power': (0.06629003928433841, 1.0656755544078196), 'Voltage': (223.69920298179903, 251.51676383904908), 'Global_intensity': (2.5391213206803798, 45.41377091776364), 'Sub_metering_1': (0.7996331696022843, 81.98030183889949), 'Sub_metering_2': (14.45730649004349, 67.41674502890318), 'Sub_metering_3': (2.756616459390673, 29.04616533324515)}, 'exact': 29030.406, 'estimate': 20304.77430941923, 'error': 8725.631690580769}
{'query': {'timestamp

### Step 4: Compute Sampling-Based Estimates and Errors for the Query Log

In [35]:
# Add estimates and errors to query log
for entry in query_log:
    entry['estimate'] = sample_sum(entry['query'], sample, full_data_size)
    entry['error'] = entry['exact'] - entry['estimate']

print("Estimates and errors computed for query log")

Estimates and errors computed for query log


Diversification

In [36]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

def diversify_query_log(query_log, k=400):
    """
    Diversify the query log using greedy max-min (paper Section 5.2).
    Features: flattened ranges + error.
    Returns a subset of k diverse entries.
    """
    # Prepare features: flatten query bounds + error
    features = []
    for entry in query_log:
        vec = []
        for dim in dimensions:
            lower, upper = entry['query'][dim]
            vec.extend([lower, upper])
        vec.append(entry['error'])
        features.append(vec)
    
    features = np.array(features)
    scaler_div = StandardScaler()
    features_norm = scaler_div.fit_transform(features)
    
    # Greedy max-min selection
    selected_indices = [random.randint(0, len(features)-1)]  # Start with random
    selected_features = features_norm[selected_indices]
    
    while len(selected_indices) < k:
        # Distances from unselected to current selected set
        dists = euclidean_distances(selected_features, features_norm)
        min_dists = dists.min(axis=0)  # Min dist to any selected
        # Pick the one with max min-dist (most diverse)
        next_idx = np.argmax(min_dists)
        selected_indices.append(next_idx)
        selected_features = features_norm[selected_indices]
    
    diversified_log = [query_log[i] for i in selected_indices]
    print(f"Diversified query log: selected {len(diversified_log)} / {len(query_log)} entries")
    return diversified_log

In [37]:
# After generating query_log
diversified_log = diversify_query_log(query_log, k=800)  # e.g., half of 800

# Use diversified_log instead of query_log for training the error model
# (prepare X, y from diversified_log, then model.fit)

Diversified query log: selected 800 / 2000 entries


In [38]:
diversified_log[200]

{'query': {'timestamp': (5102692.2068104595, 122042449.3112193),
  'Global_reactive_power': (0.3333339497969312, 1.3409068235175714),
  'Voltage': (225.28655776021898, 250.80150285755678),
  'Global_intensity': (0.7810399252718636, 48.050901670244315),
  'Sub_metering_1': (4.018068637258668, 72.19530246121946),
  'Sub_metering_2': (9.889978385546435, 63.20097718887258),
  'Sub_metering_3': (6.802914036110466, 26.689890653882063)},
 'exact': 4776.316,
 'estimate': 0.0,
 'error': 4776.316}

### Step 5: Train the Error Prediction Model

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare features (flatten: lower/upper per dim) and targets (errors)
X = []
y = []
# for entry in query_log:
for entry in diversified_log:

    vec = []
    for dim in dimensions:
        lower, upper = entry['query'][dim]
        vec.extend([lower, upper])
    X.append(vec)
    y.append(entry['error'])

X = np.array(X)
y = np.array(y)

# Normalize
scaler_div = StandardScaler()
X_scaled = scaler_div.fit_transform(X)

# Train (80/20 split for validation)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# adjust max_depth
model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train, y_train)

print(f"Model trained. Test MSE: {np.mean((model.predict(X_test) - y_test)**2)}")

Model trained. Test MSE: 5726575.324808587


### Step 6: Estimate a New Query

In [40]:
# Example new 7D query (adjust ranges to sensible values based on data mins/maxes)
new_query = {
    'timestamp': (0, 1e8),  # e.g., first ~few months in seconds
    'Global_reactive_power': (0.0, 0.5),
    'Voltage': (220, 250),
    'Global_intensity': (0, 20),
    'Sub_metering_1': (0, 10),
    'Sub_metering_2': (0, 5),
    'Sub_metering_3': (0, 15)
}

# Flatten and scale
new_vec = []
for dim in dimensions:
    lower, upper = new_query[dim]
    new_vec.extend([lower, upper])
new_vec = np.array([new_vec])
new_scaled = scaler_div.transform(new_vec)

# Predict error
predicted_error = model.predict(new_scaled)[0]

# Find error-similar historical query (closest error)
min_diff = float('inf')
opt_entry = None
for entry in diversified_log:
    error_diff = abs(entry['error'] - predicted_error)
    if error_diff < min_diff:
        min_diff = error_diff
        opt_entry = entry

# Compute final estimate
sample_new = sample_sum(new_query, sample, full_data_size)
sample_opt = opt_entry['estimate']
final_estimate = opt_entry['exact'] + (sample_new - sample_opt)

print(f"Final estimate for new query: {final_estimate}")

Final estimate for new query: 550051.2473206442


In [41]:
# After computing final_estimate
print(f"LAQP estimate: {final_estimate:.2f}")

# Compute exact for the same query (for debugging/small queries)
exact = exact_sum(new_query, data)
print(f"Exact sum: {exact:.2f}")
print(f"Relative error: {abs(final_estimate - exact) / exact:.4f}")

# Also see how many rows match
mask = np.ones(len(data), dtype=bool)
for dim, (l, u) in new_query.items():
    mask &= (data[dim] >= l) & (data[dim] <= u)
matched_rows = mask.sum()
print(f"Query matches {matched_rows:,} rows ({matched_rows / len(data):.1%} of dataset)")

LAQP estimate: 550051.25
Exact sum: 560494.03
Relative error: 0.0186
Query matches 1,059,647 rows (51.7% of dataset)


In [42]:
def laqp_estimate_with_details(query):
    # Flatten and predict error (same as before)
    vec = []
    for dim in dimensions:
        l, u = query[dim]
        vec.extend([l, u])
    vec = np.array([vec])
    scaled = scaler_div.transform(vec)
    pred_error = model.predict(scaled)[0]
    
    # Find the most error-similar historical query
    best_index = -1
    best_error_diff = float('inf')
    best_entry = None

    for idx, entry in enumerate(diversified_log):
        error_diff = abs(entry['error'] - pred_error)
        if error_diff < best_error_diff:
            best_error_diff = error_diff
            best_index = idx
            best_entry = entry
    
    # Compute estimates
    sample_new = sample_sum(query, sample, full_data_size)
    sample_opt = best_entry['estimate']
    final_est = best_entry['exact'] + (sample_new - sample_opt)
    
    print(f"Selected optimal query index: {best_index} (out of  {len(diversified_log)})")
    print(f"Predicted error for new query: {pred_error:.2f}")
    print(f"Chosen historical query error: {best_entry['error']:.2f} (diff: {best_error_diff:.2f})")
    print(f"Exact result of chosen query: {best_entry['exact']:.2f}")
    print("Predicate ranges of chosen query:")
    for dim, (l, u) in best_entry['query'].items():
        print(f"  {dim}: [{l:.2f}, {u:.2f}]")
    print(f"\nFinal LAQP estimate: {final_est:.2f}")    

    return final_est, best_index, best_entry

# Use it
estimate, opt_idx, opt_entry = laqp_estimate_with_details(new_query)

Selected optimal query index: 705 (out of  800)
Predicted error for new query: 461.43
Chosen historical query error: 436.15 (diff: 25.28)
Exact result of chosen query: 6352.96
Predicate ranges of chosen query:
  timestamp: [5144387.73, 119582258.28]
  Global_reactive_power: [0.24, 1.09]
  Voltage: [229.19, 248.42]
  Global_intensity: [8.49, 45.06]
  Sub_metering_1: [11.40, 68.26]
  Sub_metering_2: [16.59, 65.20]
  Sub_metering_3: [4.61, 28.54]

Final LAQP estimate: 550051.25


### Step 7: Evaluate and Extend
Basic Evaluation: Measure Accuracy on Test Queries

Optimization


In [43]:
def range_distance(q1, q2):
    """Euclidean distance on flattened predicate bounds (for range-similarity)."""
    vec1 = []
    vec2 = []
    for dim in dimensions:
        l1, u1 = q1[dim]
        l2, u2 = q2[dim]
        vec1.extend([l1, u1])
        vec2.extend([l2, u2])
    return np.linalg.norm(np.array(vec1) - np.array(vec2))

In [44]:
from scipy.optimize import minimize_scalar

def optimize_alpha(query_log, val_queries, model, scaler, sample, full_data_size, bounds=(0,1)):
    """
    Tune alpha for hybrid similarity (paper Section 5.3).
    val_queries: List of {'query': dict, 'exact': float} for tuning.
    Returns best alpha that minimizes average relative error on val set.
    """
    def objective(alpha):
        errors = []
        for vq in val_queries:
            query = vq['query']
            exact = vq['exact']
            
            # Predict error
            vec = [query[dim][i] for dim in dimensions for i in range(2)]
            vec = np.array([vec])
            scaled = scaler.transform(vec)
            pred_error = model.predict(scaled)[0]
            
            # Find best entry with hybrid similarity
            best_entry = min(diversified_log, key=lambda e: 
                alpha * abs(e['error'] - pred_error) + 
                (1 - alpha) * range_distance(query, e['query']))
            
            # LAQP estimate
            sample_new = sample_sum(query, sample, full_data_size)
            sample_opt = best_entry['estimate']
            laqp_est = best_entry['exact'] + (sample_new - sample_opt)
            
            # Relative error
            rel_err = abs(laqp_est - exact) / (exact + 1e-6)
            errors.append(rel_err)
        
        return np.mean(errors)
    
    # Optimize alpha
    res = minimize_scalar(objective, bounds=bounds, method='bounded')
    best_alpha = res.x
    print(f"Optimized alpha: {best_alpha:.3f} (MSE on val: {res.fun:.4f})")
    return best_alpha

In [45]:
# Generate 100 separate test queries
test_queries = []
num_test = 100

print("Generating 100 test queries...")
while len(test_queries) < num_test:
    q = generate_random_query_paper_style()
    exact_result = exact_sum(q, data)
    if exact_result > 0.01 * avg_exact:
        test_queries.append({'query': q, 'exact': exact_result})

print(f"Generated {len(test_queries)} test queries")

val_queries = test_queries[:50]
final_test_queries = test_queries[50:]
best_alpha = optimize_alpha(diversified_log, val_queries, model, scaler_div, sample, full_data_size)

# Evaluation lists
laqp_rel_errors = []
sampling_rel_errors = []
laqp_abs_errors = []
sampling_abs_errors = []

# Threshold for filtering tiny queries (adjustable)
# Good starting values: 1000 or 0.01 * average exact sum
avg_exact = np.mean([tq['exact'] for tq in final_test_queries])
min_exact_threshold = max(1000.0, 0.01 * avg_exact)  # at least 1000 or 1% of avg

print(f"Using minimum exact threshold: {min_exact_threshold:.2f} "
      f"(avg exact = {avg_exact:.2f})")

Generating 100 test queries...
Generated 100 test queries
Optimized alpha: 1.000 (MSE on val: 0.1218)
Using minimum exact threshold: 1000.00 (avg exact = 9324.23)


In [46]:
filtered_count = 0

for tq in final_test_queries:
    query = tq['query']
    exact = tq['exact']
    
    if exact < min_exact_threshold:
        continue  # skip tiny queries that distort relative error
    filtered_count += 1
    
    # --- Pure Sampling Estimate ---
    sample_est = sample_sum(query, sample, full_data_size)
    sampling_abs = abs(sample_est - exact)
    sampling_rel = sampling_abs / exact
    
    sampling_abs_errors.append(sampling_abs)
    sampling_rel_errors.append(sampling_rel)
    
    # --- LAQP Estimate ---
    # Flatten and predict error
    vec = []
    for dim in dimensions:
        l, u = query[dim]
        vec.extend([l, u])
    vec = np.array([vec])
    scaled = scaler.transform(vec)
    predicted_error = model.predict(scaled)[0]
    
    # Hybrid selection
    best_entry = min(diversified_log, key=lambda e: 
        best_alpha * abs(e['error'] - predicted_error) + 
        (1 - best_alpha) * range_distance(query, e['query']))
    
    # Compute estimates
    sample_new = sample_sum(query, sample, full_data_size)
    sample_opt = best_entry['estimate']
    laqp_est = best_entry['exact'] + (sample_new - sample_opt)
    
    laqp_abs = abs(laqp_est - exact)
    laqp_rel = laqp_abs / exact
    
    laqp_abs_errors.append(laqp_abs)
    laqp_rel_errors.append(laqp_rel)

# ========================
# Results
# ========================
print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
print(f"Evaluated on {filtered_count} / {len(final_test_queries)} queries "
      f"(excluded {len(final_test_queries)-filtered_count} tiny ones)")

if len(laqp_rel_errors) > 0:
    print(f"\nLAQP    Average Relative Error (ARE): {np.mean(laqp_rel_errors):.4f}")
    print(f"LAQP    Median Relative Error:       {np.median(laqp_rel_errors):.4f}")
    print(f"LAQP    Mean Absolute Error (MAE):     {np.mean(laqp_abs_errors):.2f}")
    
    print(f"\nSampling ARE:                       {np.mean(sampling_rel_errors):.4f}")
    print(f"Sampling Median Relative Error:     {np.median(sampling_rel_errors):.4f}")
    print(f"Sampling MAE:                       {np.mean(sampling_abs_errors):.2f}")
    
    improvement = np.mean(sampling_rel_errors) / np.mean(laqp_rel_errors)
    print(f"\nImprovement (Sampling ARE / LAQP ARE): {improvement:.2f}x")
else:
    print("No queries passed the threshold — try lowering min_exact_threshold")


EVALUATION RESULTS
Evaluated on 50 / 50 queries (excluded 0 tiny ones)

LAQP    Average Relative Error (ARE): 0.1111
LAQP    Median Relative Error:       0.0909
LAQP    Mean Absolute Error (MAE):     1037.98

Sampling ARE:                       0.7029
Sampling Median Relative Error:     0.8234
Sampling MAE:                       5215.47

Improvement (Sampling ARE / LAQP ARE): 6.33x
